In [1]:
from ilqr import iLQR
import gym
import numpy as np
import pandas as pd
import time

from aprl.agents import MujocoFiniteDiffDynamicsPerformance
from aprl.envs.mujoco_costs import get_cost
from experiments.common import set_seeds, make_env, fit_ilqr, \
                               on_iteration, receding, multi_evaluate

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


Logging to /tmp/openai-2019-02-02-10-04-05-806749
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']


In [6]:
seed = 42
set_seeds(seed)

env_name = 'InvertedPendulum-v2'
env, us_init = make_env(env_name, seed=seed)
dynamics = {'performance': MujocoFiniteDiffDynamicsPerformance(env)}
x0s = {k: dyn.get_state() for k, dyn in dynamics.items()}

costs = {k: get_cost(env_name) for k in dynamics.keys()}
ilqrs = {k: iLQR(dyn, costs[k], len(us_init))  
         for k, dyn in dynamics.items()}

In [7]:
xs, us = fit_ilqr(ilqrs, x0s, us_init)

dict_keys(['performance'])
*** Fitting performance ***
iteration -1 converged 7046.007170758981 [-9.91327735e-01  1.57328881e+00  7.48580244e-01 -7.60584215e-05] [1.1844022]
iteration 0 accepted 3490.9683057707293 [-1.01265828e+00  1.57328817e+00  1.62492151e-01 -8.32966987e-05] [0.02293796]
iteration 1 accepted 3436.987237471468 [-9.66623384e-01  1.57327651e+00 -1.08633958e+00  9.90975118e-08] [0.0086602]
iteration 2 accepted 3281.722105547462 [ 9.90486799e-01  1.57327631e+00 -8.85850059e-03 -2.60718145e-07] [-0.00816767]
iteration 3 accepted 3257.141189971252 [ 9.92193492e-01  1.57327635e+00 -1.83838362e-05  4.82276185e-12] [-7.1889945e-06]
iteration 4 accepted 3203.0896623830226 [ 9.90973547e-01  1.57327635e+00 -8.79031095e-06  2.00262260e-10] [1.2269153e-06]
iteration 5 accepted 3145.771453748645 [ 9.90698211e-01  1.57327635e+00 -1.00114855e-05  1.19123047e-10] [1.3982362e-06]
iteration 6 accepted 3129.4141752051905 [ 9.90792578e-01  1.57327635e+00 -1.20267217e-05 -1.45479271e-11] 

iteration 67 failed 2337.5773966922434 [-9.97786720e-01  1.57327634e+00  1.94887791e-02 -8.20614657e-07] [-0.00272185]
iteration 68 failed 2337.5773966922434 [-9.97786720e-01  1.57327634e+00  1.94887791e-02 -8.20614657e-07] [-0.00272185]
iteration 69 failed 2337.5773966922434 [-9.97786720e-01  1.57327634e+00  1.94887791e-02 -8.20614657e-07] [-0.00272185]
iteration 70 failed 2337.5773966922434 [-9.97786720e-01  1.57327634e+00  1.94887791e-02 -8.20614657e-07] [-0.00272185]
iteration 71 failed 2337.5773966922434 [-9.97786720e-01  1.57327634e+00  1.94887791e-02 -8.20614657e-07] [-0.00272185]
iteration 72 accepted 2326.5643600129515 [-1.00038098e+00  1.57327730e+00  3.20213138e-02 -1.70859892e-05] [-0.00296162]
iteration 73 accepted 2209.155323169551 [-1.00436917e+00  1.57327929e+00  3.40264118e-02  1.38222433e-05] [0.00195659]
iteration 74 accepted 2204.1826755699744 [-9.69532851e-01  1.57327637e+00 -2.79648076e-01  4.94675375e-07] [0.00459934]
iteration 75 accepted 2170.3349540526888 [ 9.

# Receding horizon

In [ ]:
for k, ilqr in ilqrs.items():
    underlying = 'receding_' + k
    dynamics[underlying] = dynamics[k]
    x0s[underlying] = x0s[k]
    xs[underlying], us[underlying] = receding(ilqr, x0s[k], us_init, seed=seed)

# Rollouts

In [8]:
multi_evaluate(env, dynamics, x0s, us)

performance


,rewards,lengths
performance,289.0,289
